# Laboratorium 4
#### Odległość edycyjna

In [1]:
from unidecode import unidecode
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
from copy import deepcopy
from random import randint

1. Zaimplementuj algorytm liczący odległość edycyjną i umożliwiający odzyskanie sekwencji operacji.

In [2]:
def delta(a, b):
    if a==b:
        return 0.
    elif unidecode(a)==unidecode(b):
        return 0.5
    else:
        return 1.
    

def Leven(A, B, ver=False, delta=delta, ed=False):
    edit = [[0. for i in range(len(B)+1)] for j in range(len(A)+1)]
    for i in range(len(B)):
        edit[0][i+1] = i+1.
    for i in range(len(A)):
        edit[i+1][0] = i+1.
    for i in range(len(A)):
        for j in range(len(B)):
            edit[i+1][j+1] = min(edit[i][j]+delta(A[i], B[j]), edit[i][j+1]+1, edit[i+1][j]+1)
    if ver:
        print('\n'.join(', '.join(map(str,sl)) for sl in edit))
    if ed:
        return edit[-1][-1], edit
    ops = []
    ix = len(A)
    jx = len(B)
    while ix>0 and jx>0:
        mi = min(edit[ix-1][jx-1], edit[ix-1][jx], edit[ix][jx-1])
        if edit[ix-1][jx-1] == mi:
            dlt = delta(A[ix-1], B[jx-1])
            if dlt != 0:
                ops.append(3)
            else:
                ops.append(0)
            ix-=1
            jx-=1
        elif edit[ix-1][jx] == mi:
            ops.append(1)
            ix-=1
        else:
            ops.append(2)
            jx-=1
            
    while ix>0:
        ops.append(1)
        ix-=1
    while jx>0:
        ops.append(2)
        jx-=1
    ops.reverse() # 0 - equal, 1 - add A, 2 - add B, 3 - change A -> B
    return edit[-1][-1], ops

In [3]:
# Leven('wojtk', 'wjeek', True)
Leven('los', 'kloc', True)

0.0, 1.0, 2.0, 3.0, 4.0
1.0, 1.0, 1.0, 2.0, 3.0
2.0, 2.0, 2.0, 1.0, 2.0
3.0, 3.0, 3.0, 2.0, 2.0


(2.0, [2, 0, 0, 3])

2.Zaimplementuj wizualizację.

In [4]:
def vis(A, B, ops):
    if isinstance(ops, tuple):
        _, ops = ops
    cnt = 1
    res = ''
    ix = 0
    jx = 0
    print("Distance", _)
    print(A)
    for op in ops:
        if op == 1:
            print(f"{cnt}. {res}_{A[ix+1:] if ix+1<len(A) else ''}")
            cnt += 1
            jx -= 1
        elif op == 2:
            print(f"{cnt}. {res}+{B[jx]}+{A[ix:] if ix<len(A) else ''}")
            res += B[jx]
            cnt += 1
            ix -= 1
        elif op == 3:
            print(f"{cnt}. {res}*{B[jx]}*{A[ix+1:] if ix+1<len(A) else ''}")
            res += B[jx]
            cnt += 1
        else:
            res += A[ix]
        ix += 1
        jx += 1
    print(res)
    print("Correct:", res==B)

In [5]:
vis('wojtk', 'wjeek', Leven('wojtk', 'wjeek'))

Distance 3.0
wojtk
1. w*j*jtk
2. wj*e*tk
3. wje*e*k
wjeek
Correct: True


3. Przedstaw wynik działania dla podanych przykładów.

In [6]:
vis('los', 'kloc', Leven('los', 'kloc'))
print("\n")
vis('Łódź', 'Lodz', Leven('Łódź', 'Lodz'))
print("\n")
vis('kwintesencja', 'quintessence', Leven('kwintesencja', 'quintessence'))
print("\n")
vis('ATGAATCTTACCGCCTCG', 'ATGAGGCTCTGGCCCCTG', Leven('ATGAATCTTACCGCCTCG', 'ATGAGGCTCTGGCCCCTG'))

Distance 2.0
los
1. +k+los
2. klo*c*
kloc
Correct: True


Distance 1.5
Łódź
1. *L*ódź
2. L*o*dź
3. Lod*z*
Lodz
Correct: True


Distance 5.0
kwintesencja
1. *q*wintesencja
2. q*u*intesencja
3. quintes+s+encja
4. quintessenc_a
5. quintessenc*e*
quintessence
Correct: True


Distance 7.0
ATGAATCTTACCGCCTCG
1. ATGA*G*TCTTACCGCCTCG
2. ATGAG*G*CTTACCGCCTCG
3. ATGAGGCT+C+TACCGCCTCG
4. ATGAGGCTCT*G*CCGCCTCG
5. ATGAGGCTCTG*G*CGCCTCG
6. ATGAGGCTCTGGC*C*CCTCG
7. ATGAGGCTCTGGCCCCT_G
ATGAGGCTCTGGCCCCTG
Correct: True


4. Zaimplementuj LCS - najdłuższy wspólny podciąg dla pary ciągów elementów.

In [7]:
def LCS(A, B):
    res, edit = Leven(A, B, ed=True, delta=(lambda x, y: 0 if x==y else 2))
    res = (len(A)+len(B)-res)//2
    lcs = []
    ix = len(A)
    jx = len(B)
    while ix>0 and jx>0:
        if A[ix-1]==B[jx-1]:
            lcs += [A[ix-1]]
            ix -= 1
            jx -= 1
        elif edit[ix-1][jx] < edit[ix][jx-1]:
            ix-=1
        else:
            jx-=1
    lcs.reverse()
    return res, lcs

In [8]:
LCS(list('wojtk'), list('wjeek'))
# LCS(list('kwintesencja'), list('quintessence'))

(3.0, ['w', 'j', 'k'])

5. Korzystając z tokenizera dokonaj podziału załączonego tekstu na tokeny.

In [9]:
def tokenize(name="romeo-i-julia-700.txt"):
    tokenizer = Tokenizer(Polish().vocab)
    rnj=[]
    with open(name, 'r') as file:
        rnj = file.readlines()
    for i in range(len(rnj)):
        rnj[i] = tokenizer(rnj[i])
#     return list(map(str, rnj))
    return rnj

In [10]:
tokenized = tokenize()

6. Stwórz 2. wersje załączonego tekstu, w których usunięto 3% losowych tokenów.

In [11]:
def randomize(inps):
    tokenized = deepcopy(inps)
    size = 0
    for i in range(len(tokenized)):
        size += len(tokenized[i])
    redun = int((size*3)/100)
    for i in range(redun):
        row = randint(0, len(tokenized)-1)
        while len(tokenized[row])<3:
            row = randint(0, len(tokenized)-1)
        col = randint(1, len(tokenized[row])-2)
        tokenized[row] = [tokenized[row][:col]]+[tokenized[row][col+1:]]
    return tokenized

In [12]:
tokenized1 = randomize(tokenized)
tokenized2 = randomize(tokenized)

In [13]:
def savetofile(tokenized, name="tokenized1.txt"):
    with open(name, 'w') as file:
        for line in tokenized:
            file.write(" ".join(list(map(str,line))))

In [14]:
savetofile(tokenized1, "tokenized1.txt")
savetofile(tokenized2, "tokenized2.txt")

7. Oblicz LCS wspólnych tokenów dla tych tekstów.

In [15]:
def toklcs(A="tokenized1.txt", B="tokenized2.txt"):
    tokenizer = Tokenizer(Polish().vocab)
    with open(A, 'r') as file:
        tokenized1 = tokenizer(file.read())
    with open(B, 'r') as file:
        tokenized2 = tokenizer(file.read())
    return LCS(list(map(str, tokenized1)), list(map(str, tokenized2)))

In [16]:
res, lcs = toklcs()
print(res)
print(" ".join(lcs)[:126])

2112.0
William Shakespeare 

 Romeo i Julia 
 tłum. Józef 

 ISBN 978-83-288-2903-9 



 OSOBY: 
   * ESKALUS — książę panujący w Wer


8. Korzystając z algorytmu z punktu 4. skonstruuj diff - wskazywanie w dwóch plikach linie które się różnią. Na wyjściu narzędzia powinny znaleźć się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać skąd dana linia pochodzi (< > - pierwszy/drugi plik) oraz numer linii w danym pliku.

In [17]:
def endlines(a):
    return a.count('\n')

In [18]:
def finddiffs(tokenized, lcs):
    pos = 0
    line = 0
    res = []
    for ix in range(len(tokenized)):
        line += endlines(tokenized[ix])
        if tokenized[ix] != lcs[pos]:
            res.append(line)
        else:
            pos+=1
    return res

In [19]:
def diff(A="tokenized1.txt", B="tokenized2.txt"):
    tokenizer = Tokenizer(Polish().vocab)
    with open(A, 'r') as file:
        lines1 = file.readlines()
    with open(A, 'r') as file:
        tokenized1 = tokenizer(file.read())
    with open(B, 'r') as file:
        lines2 = file.readlines()
    with open(B, 'r') as file:
        tokenized2 = tokenizer(file.read())
    tokenized1 = list(map(str, tokenized1))
    tokenized2 = list(map(str, tokenized2))
    _, lcs = LCS(tokenized1, tokenized2)

    res1 = finddiffs(tokenized1, lcs)
    res2 = finddiffs(tokenized2, lcs)
    
    se = sorted(list(set(res1 + res2)))
    for item in se:
        print("\nLine", item)
        print(f"{len(lines1[item])}<{lines1[item]}")
        print(f"{len(lines2[item])}>{lines2[item]}")

9. Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

In [20]:
diff()


Line 3
13<tłum. Józef 

24>tłum. Józef Paszkowski 


Line 11
57< * — młody Weroneńczyk szlachetnego rodu, krewny księcia

65>  * PARYS — młody Weroneńczyk szlachetnego rodu, krewny księcia 


Line 17
25< * TYBALT — Pani Kapulet

34>  * TYBALT — krewny Pani Kapulet 


Line 18
26< * — ojciec franciszkanin

37>  * LAURENTY — ojciec franciszkanin 


Line 26
11<  * PIOTR 

8>  PIOTR


Line 28
36< * PANI MONTEKI małżonka Montekiego

33> * MONTEKI — małżonka Montekiego


Line 30
29<  * JULIA — córka Kapuletów 

25> * JULIA córka Kapuletów


Line 31
25<  * MARTA — mamka Julii 

17> * — mamka Julii


Line 37
86<Rzecz odbywa przez większą część sztuki w Weronie, przez część piątego aktu w Mantui.

91>Rzecz odbywa się przez większą część sztuki w Weronie, przez część piątego aktu w Mantui. 


Line 45
28<Dwa rody, zacne jednako i —

27>Dwa rody, zacne i sławne —


Line 51
37<Pod najstraszliwszą z gwiazd, dwoje;

48>Pod najstraszliwszą z gwiazd, kochanków dwoje; 


Line 55
29<Tej ich miłości zbyt 